# UQ MDS - Assessment administration

## Required
The cell below must always be run when launching a new Jupyter notebook instance.<br />
Specify the relative path of the course configuration file as a parameter.

In [ ]:
from ghca.gitlink import GitHubLink
GitHubLink = GitHubLink('config/TEST1000_2018_S1.json')

## Assign students to assessment (group work)
Imports the contents of a CSV file to establish group members. <br />

* The method can be ignored if the assessment was configured with `max-group-size = 1`. 
* Students that are not included in the CSV import will be treated as doing individual work when the assessment is prepared.
* Creates groups.json in the `main-dir` of the assessment directory (in the instructors repository).
* Imports data based on a CSV which expects the header format GroupName|Members where members are the comma separated GitHub usernames of each student.
* The group name will be used when creating the repository name and therefore must be unique for each assessment
* If group name is None then the students username will be used for groups n=1.
* If this is not a group assessment, run it with anything for input_csv. The script will automatically determine the maximum group size based off the assessment_config.json file and create singleton groups.
* Max group size of 0 = unlimited

In [ ]:
IMPORT_STUDENT_GROUPS_CSV = """GroupName|Members
group1|test001,test002
test003|test003
"""

GitHubLink.import_assessment_groups_csv(name_assessment='a1', csv_input=IMPORT_STUDENT_GROUPS_CSV)

## Prepare assessment
Creates a repository for each student containing the assessment files.
* FILES MUST BE ADDED TO THE INSTRUCTORS REPO BEFORE CALLING
* Visible to instructors
* Invisible to students until release_assessment is called.
* Students not in a group will be automatically added to the assessment as doing individual work.

In [ ]:
GitHubLink.prepare_assessment(name_assessment='a1', name_target_branch='master', overwrite=False)

## Release assessment
Invites students to a given assessment repository. Available student permissions:
* push = Read/write
* pull = Read
* Admin = full control

In [ ]:
GitHubLink.release_assessment(name_assessment='a1', permission='push')

## Update assessment
To be used if an item of assessment has been updated after it has been released. 
* Calls release_assessment to transfer new files to the instructor_updates branch for each students assessment repository.
* Creates a pull request notifying the student to merge the instructor_updates branch with master.

In [ ]:
GitHubLink.update_assessment_pr(name_assessment='a1')

## Close assessment
To be used to close students assessment when due.
* Sets student access to pull (read) on their repository

In [ ]:
GitHubLink.close_assessment(name_assessment='a1', compress=True)

## Forfeit assessment
To be used to release an item of assessment back to the user. Helps in reducing the amount of repositories in the organisation.

* Assigns admin rights to each member of the repository.
* Messages the users to request they transfer ownership to themselves.

In [ ]:
GitHubLink.forfeit_assessment('a1')